Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/work-with-data/datasets-tutorial/train-with-datasets.png)

# Train with Azure Machine Learning Datasets
Datasets are categorized into TabularDataset and FileDataset based on how users consume them in training. 
* A TabularDataset represents data in a tabular format by parsing the provided file or list of files. TabularDataset can be created from csv, tsv, parquet files, SQL query results etc. For the complete list, please visit our [documentation](https://aka.ms/tabulardataset-api-reference). It provides you with the ability to materialize the data into a pandas DataFrame.
* A FileDataset references single or multiple files in your datastores or public urls. This provides you with the ability to download or mount the files to your compute. The files can be of any format, which enables a wider range of machine learning scenarios including deep learning.

In this tutorial, you will learn how to train with Azure Machine Learning Datasets:

&#x2611; Use Datasets directly in your training script

&#x2611; Use Datasets to mount files to a remote compute

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [configuration notebook](../../../configuration.ipynb) first if you haven't already established your connection to the AzureML Workspace.

In [ ]:
# Check core SDK version number
import azureml.core

print('SDK version:', azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Create Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
experiment_name = 'train-with-datasets'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

## Create or Attach existing compute resource
By using Azure Machine Learning Compute, a managed service, data scientists can train machine learning models on clusters of Azure virtual machines. Examples include VMs with GPU support. In this tutorial, you create Azure Machine Learning Compute as your training environment. The code below creates the compute clusters for you if they don't already exist in your workspace.

**Creation of compute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace the code will skip the creation process.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get('AML_COMPUTE_CLUSTER_NAME', 'cpu-cluster')
compute_min_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MIN_NODES', 0)
compute_max_nodes = os.environ.get('AML_COMPUTE_CLUSTER_MAX_NODES', 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get('AML_COMPUTE_CLUSTER_SKU', 'STANDARD_D2_V2')


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes, 
                                                                max_nodes=compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

You now have the necessary packages and compute resources to train a model in the cloud.
## Use Datasets directly in training

### Create a TabularDataset
By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred. 

Every workspace comes with a default [datastore](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-access-data) (and you can register more) which is backed by the Azure blob storage account associated with the workspace. We can use it to transfer data from local to the cloud, and create Dataset from it. We will now upload the [Iris data](./train-dataset/Iris.csv) to the default datastore (blob) within your workspace.

In [ ]:
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train-dataset/iris.csv'],
                       target_path = 'train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)

Then we will create an unregistered TabularDataset pointing to the path in the datastore. You can also create a Dataset from multiple paths. [learn more](https://aka.ms/azureml/howto/createdatasets) 

In [ ]:
from azureml.core import Dataset
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/iris.csv')])

# preview the first 3 rows of the dataset
dataset.take(3).to_pandas_dataframe()

### Create a training script

To submit the job to the cluster, first create a training script. Run the following code to create the training script called `train_titanic.py` in the script_folder. 

In [ ]:
script_folder = os.path.join(os.getcwd(), 'train-dataset')

In [ ]:
%%writefile $script_folder/train_iris.py

import os

from azureml.core import Dataset, Run
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals import joblib

run = Run.get_context()
# get input dataset by name
dataset = run.input_datasets['iris']

df = dataset.to_pandas_dataframe()

x_col = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
y_col = ['species']
x_df = df.loc[:, x_col]
y_df = df.loc[:, y_col]

#dividing X,y into train and test data
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)

data = {'train': {'X': x_train, 'y': y_train},

        'test': {'X': x_test, 'y': y_test}}

clf = DecisionTreeClassifier().fit(data['train']['X'], data['train']['y'])
model_file_name = 'decision_tree.pkl'

print('Accuracy of Decision Tree classifier on training set: {:.2f}'.format(clf.score(x_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(clf.score(x_test, y_test)))

os.makedirs('./outputs', exist_ok=True)
with open(model_file_name, 'wb') as file:
    joblib.dump(value=clf, filename='outputs/' + model_file_name)

### Configure and use Datasets as the input to Estimator

You can ask the system to build a conda environment based on your dependency specification. Once the environment is built, and if you don't change your dependencies, it will be reused in subsequent runs.

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

conda_env = Environment('conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(pip_packages=['azureml-sdk',
                                                                             'azureml-dataprep[pandas,fuse]',
                                                                             'scikit-learn'])

An estimator object is used to submit the run. Azure Machine Learning has pre-configured estimators for common machine learning frameworks, as well as generic Estimator. Create a generic estimator for by specifying

* The name of the estimator object, `est`
* The directory that contains your scripts. All the files in this directory are uploaded into the cluster nodes for execution. 
* The training script name, train_titanic.py
* The input Dataset for training
* The compute target. In this case you will use the AmlCompute you created
* The environment definition for the experiment

In [ ]:
from azureml.train.estimator import Estimator

est = Estimator(source_directory=script_folder, 
                entry_script='train_iris.py', 
                # pass dataset object as an input with name 'titanic'
                inputs=[dataset.as_named_input('iris')],
                compute_target=compute_target,
                environment_definition= conda_env) 

### Submit job to run
Submit the estimator to the Azure ML experiment to kick off the execution.

In [ ]:
run = exp.submit(est)

In [ ]:
from azureml.widgets import RunDetails

# monitor the run
RunDetails(run).show()

## Use Datasets to mount files to a remote compute

You can use the Dataset object to mount or download files referred by it. When you mount a file system, you attach that file system to a directory (mount point) and make it available to the system. Because mounting load files at the time of processing, it is usually faster than download.<br> 
Note: mounting is only available for Linux-based compute (DSVM/VM, AMLCompute, HDInsights).

### Upload data files into datastore
We will first load diabetes data from `scikit-learn` to the train-dataset folder.

In [ ]:
from sklearn.datasets import load_diabetes
import numpy as np

training_data = load_diabetes()
np.save(file='train-dataset/features.npy', arr=training_data['data'])
np.save(file='train-dataset/labels.npy', arr=training_data['target'])

Now let's upload the 2 files into the default datastore under a path named `diabetes`:

In [ ]:
datastore.upload_files(['train-dataset/features.npy', 'train-dataset/labels.npy'], target_path='diabetes', overwrite=True)

### Create a FileDataset

In [ ]:
from azureml.core import Dataset

dataset = Dataset.File.from_files(path = [(datastore, 'diabetes/')])

# see a list of files referenced by dataset
dataset.to_path()

### Create a training script

To submit the job to the cluster, first create a training script. Run the following code to create the training script called `train_diabetes.py` in the script_folder. 

In [ ]:
%%writefile $script_folder/train_diabetes.py

import os
import glob

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from azureml.core.run import Run
from sklearn.externals import joblib

import numpy as np

os.makedirs('./outputs', exist_ok=True)

run = Run.get_context()
base_path = run.input_datasets['diabetes']

X = np.load(glob.glob(os.path.join(base_path, '**/features.npy'), recursive=True)[0])
y = np.load(glob.glob(os.path.join(base_path, '**/labels.npy'), recursive=True)[0])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)
data = {'train': {'X': X_train, 'y': y_train},
        'test': {'X': X_test, 'y': y_test}}

# list of numbers from 0.0 to 1.0 with a 0.05 interval
alphas = np.arange(0.0, 1.0, 0.05)

for alpha in alphas:
    # use Ridge algorithm to create a regression model
    reg = Ridge(alpha=alpha)
    reg.fit(data['train']['X'], data['train']['y'])

    preds = reg.predict(data['test']['X'])
    mse = mean_squared_error(preds, data['test']['y'])
    run.log('alpha', alpha)
    run.log('mse', mse)

    model_file_name = 'ridge_{0:.2f}.pkl'.format(alpha)
    with open(model_file_name, 'wb') as file:
        joblib.dump(value=reg, filename='outputs/' + model_file_name)

    print('alpha is {0:.2f}, and mse is {1:0.2f}'.format(alpha, mse))

### Configure & Run

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_folder, 
                      script='train_diabetes.py', 
                      # to mount the dataset on the remote compute and pass the mounted path as an argument to the training script
                      arguments =[dataset.as_named_input('diabetes').as_mount('tmp/dataset')])

src.run_config.framework = 'python'
src.run_config.environment = conda_env
src.run_config.target = compute_target.name

In [ ]:
run = exp.submit(config=src)

# monitor the run
RunDetails(run).show()

### Display run results
You now have a model trained on a remote cluster. Retrieve all the metrics logged during the run, including the accuracy of the model:

In [ ]:
print(run.get_metrics())
metrics = run.get_metrics()

### Register Datasets
Use the register() method to register datasets to your workspace so they can be shared with others, reused across various experiments, and referred to by name in your training script.

In [ ]:
dataset = dataset.register(workspace = ws,
                           name = 'diabetes dataset',
                           description='training dataset',
                           create_new_version=True)

## Register models with Datasets
The last step in the training script wrote the model files in a directory named `outputs` in the VM of the cluster where the job is executed. `outputs` is a special directory in that all content in this directory is automatically uploaded to your workspace. This content appears in the run record in the experiment under your workspace. Hence, the model file is now also available in your workspace.

You can register models with Datasets for reproducibility and auditing purpose.

In [ ]:
# find the index where MSE is the smallest
indices = list(range(0, len(metrics['mse'])))
min_mse_index = min(indices, key=lambda x: metrics['mse'][x])

print('When alpha is {1:0.2f}, we have min MSE {0:0.2f}.'.format(
    metrics['mse'][min_mse_index], 
    metrics['alpha'][min_mse_index]
))

In [ ]:
# find the best model
best_alpha = metrics['alpha'][min_mse_index]
model_file_name = 'ridge_{0:.2f}.pkl'.format(best_alpha)

In [ ]:
# register the best model with the input dataset
model = run.register_model(model_name='sklearn_diabetes', model_path=os.path.join('outputs', model_file_name),
                           datasets =[('training data',dataset)])